# Get Big cutout from a series of calexp into fits files from sources Light Curve

- author Sylvie Dagoret-Campagne
- creation date 2024-05-29
- last update 2024-06-07
- affiliation : IJCLab
- Kernel **w_2024_16**

- want to see within 30 arcsec from main source thus need box size of 300 pixel

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm,SymLogNorm
from matplotlib.patches import Circle,Annulus
props = dict(boxstyle='round', facecolor=None, alpha=0.1)
#props = dict(boxstyle='round')


import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.io import fits
from astropy.wcs import WCS


import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option('display.max_rows', 100)

import matplotlib.ticker                         # here's where the formatter is
import os
import re
import pandas as pd
import pickle
from collections import OrderedDict

plt.rcParams["figure.figsize"] = (4,3)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'

import scipy

In [ ]:
import gc

In [ ]:
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.time import Time


In [ ]:
import lsst.daf.butler as dafButler
#import lsst.summit.utils.butlerUtils as butlerUtils

In [ ]:
import lsst.afw.image as afwImage
import lsst.afw.display as afwDisplay
import lsst.afw.table as afwTable
import lsst.afw.display.rgb as afwRgb
import lsst.afw.image as afwImage
import lsst.geom as geom

#import lsst.sphgeom
from  lsst.geom import SpherePoint, Angle
import lsst.sphgeom

In [ ]:

from astropy.wcs import WCS
from astropy.visualization import make_lupton_rgb
import gc

import lsst.afw.display as afwDisplay
from lsst.afw.image import MultibandExposure

In [ ]:
# LSST Display
import lsst.afw.display as afwDisplay
afwDisplay.setDefaultBackend('matplotlib')

In [ ]:
transform = AsinhStretch() + PercentileInterval(99.)

In [ ]:
# INSERT YOUR collection and tract
# for rehearsal use collection 2 which have CCDvisit
butlerRoot = "/repo/embargo"

collection1 = 'LSSTComCamSim/runs/nightlyvalidation/20240402/d_2024_03_29/DM-43612'
collection2 = 'LSSTComCamSim/runs/nightlyvalidation/20240403/d_2024_03_29/DM-43612'
collection3 = 'LSSTComCamSim/runs/nightlyvalidation/20240404/d_2024_03_29/DM-43612'
collection = 'LATISS/runs/AUXTEL_DRP_IMAGING_20230509_20240414/w_2024_15/PREOPS-5069' # COMPLETED

collectionn = collection
#collections = [collection1,collection2,collection3]
collections = [collection]
collectionStr = collectionn.replace("/", "_")
fn_ccdVisit_tracts_patches = f"ccdVisittractpatch_{collectionStr}.csv"
instrument = 'LATISS'
skymapName = "latiss_v1"
where_clause = "instrument = \'" + instrument+ "\'"
tract = 3864 # mostly for light-curves
patch_sel = 236
band = 'g'
#tract = 5615
# tract = 5634 # interesting to view calib parameters
suptitle = collectionStr + f" inst = {instrument} tract = {tract}"

In [ ]:
#dataId = {"skymap": "latiss_v1", "tract": 5615, "instrument": "LATISS"}
dataId = {"skymap": skymapName, "tract": tract, "instrument": instrument}
repo = '/sdf/group/rubin/repo/oga/'
butler = dafButler.Butler(repo)
#t = Butler.get(table_sel, dataId=dataId, collections=collections)
registry = butler.registry

In [ ]:
skymap_auxtel ='latiss_v1'
skymap = butler.get('skyMap', skymap=skymap_auxtel, collections=collections)

In [ ]:
def remove_figure(fig):
    """
    Remove a figure to reduce memory footprint.

    Parameters
    ----------
    fig: matplotlib.figure.Figure
        Figure to be removed.

    Returns
    -------
    None
    """
    # get the axes and clear their images
    for ax in fig.get_axes():
        for im in ax.get_images():
            im.remove()
    fig.clf()       # clear the figure
    plt.close(fig)  # close the figure
    gc.collect()    # call the garbage collector

In [ ]:
def get_corners_radec(wcs, bbox):
    """
    Return the corners in RA,Dec in degrees given the WCS and bounding box for an image.

    Parameters
    ----------
    wcs: image WCS returned by the Butler
    bbox: bounding box returned by the Butler

    Returns
    -------
    corners_ra, corners_dec in decimal degrees
    """

    corners_x = [bbox.beginX, bbox.beginX, bbox.endX, bbox.endX]
    corners_y = [bbox.beginY, bbox.endY, bbox.endY, bbox.beginY]
    corners_ra = []
    corners_dec = []
    for i in range(4):
        radec = wcs.pixelToSky(corners_x[i], corners_y[i])
        corners_ra.append(radec.getRa().asDegrees())
        corners_dec.append(radec.getDec().asDegrees())
    
    return corners_ra, corners_dec

In [ ]:
def convert_fluxtomag(x) :
    """
    The object and source catalogs store only fluxes. There are hundreds of flux-related columns, 
    and to store them also as magnitudes would be redundant, and a waste of space.
    All flux units are nanojanskys. The AB Magnitudes Wikipedia page provides a concise resource 
    for users unfamiliar with AB magnitudes and jansky fluxes. To convert to AB magnitudes use:
    As demonstrated in Section 2.3.2, to add columns of magnitudes after retrieving columns of flux, users can do this:
    results_table['r_calibMag'] = -2.50 * numpy.log10(results_table['r_calibFlux']) + 31.4
    results_table['r_cModelMag'] = -2.50 * numpy.log10(results_table['r_cModelFlux']) + 31.4
    (from DP0 tutorial)
    """
    return -2.50 * np.log10(x) + 31.4

In [ ]:
def cutout_coadd(butler, ra, dec, band='r', datasetType='deepCoadd',
                 skymap=None, cutoutSideLength=51, **kwargs):
    """
    Produce a cutout from a coadd at the given ra, dec position.

    Adapted from DC2 tutorial notebook by Michael Wood-Vasey.

    Parameters
    ----------
    butler: lsst.daf.persistence.Butler
        Helper object providing access to a data repository
    ra: float
        Right ascension of the center of the cutout, in degrees
    dec: float
        Declination of the center of the cutout, in degrees
    band: string
        Filter of the image to load
    datasetType: string ['deepCoadd']
        Which type of coadd to load.  Doesn't support 'calexp'
    skymap: lsst.afw.skyMap.SkyMap [optional]
        Pass in to avoid the Butler read.  Useful if you have lots of them.
    cutoutSideLength: float [optional]
        Size of the cutout region in pixels.

    Returns
    -------
    MaskedImage
    """
    radec = geom.SpherePoint(ra, dec, geom.degrees)
    cutoutSize = geom.ExtentI(cutoutSideLength, cutoutSideLength)

    if skymap is None:
        skymap = butler.get("skyMap")

    # Look up the tract, patch for the RA, Dec
    tractInfo = skymap.findTract(radec)
    patchInfo = tractInfo.findPatch(radec)
    xy = geom.PointI(tractInfo.getWcs().skyToPixel(radec))
    bbox = geom.BoxI(xy - cutoutSize // 2, cutoutSize)
    patch = tractInfo.getSequentialPatchIndex(patchInfo)

    coaddId = {'tract': tractInfo.getId(), 'patch': patch, 'band': band}
    parameters = {'bbox': bbox}

    cutout_image = butler.get(datasetType, parameters=parameters,
                              dataId=coaddId)

    return cutout_image

In [ ]:
def cutout_calexp(butler, ra, dec, visit, detector, cutoutSideLength=51, **kwargs):
    
    """
    Produce a cutout from a calexp at the given ra, dec position.

    Adapted from cutout_coadd which was adapted from a DC2 tutorial
    notebook by Michael Wood-Vasey.

    Parameters
    ----------
    butler: lsst.daf.persistence.Butler
        Helper object providing access to a data repository
    ra: float
        Right ascension of the center of the cutout, in degrees
    dec: float
        Declination of the center of the cutout, in degrees
    visit: int
        Visit id of the calexp's visit
    detector: int
        Detector for the calexp
    cutoutSideLength: float [optional]
        Size of the cutout region in pixels.

    Returns
    -------
    MaskedImage
    """
    
    dataId = {'visit': visit, 'detector': detector}    
    radec = geom.SpherePoint(ra, dec, geom.degrees)
    cutoutSize = geom.ExtentI(cutoutSideLength, cutoutSideLength)    
    calexp_wcs = butler.get('calexp.wcs', **dataId)
    xy = geom.PointI(calexp_wcs.skyToPixel(radec))
    bbox = geom.BoxI(xy - cutoutSize // 2, cutoutSize)
    parameters = {'bbox': bbox}
    cutout_image = butler.get('calexp', parameters=parameters, **dataId)

    return cutout_image

In [ ]:
def create_rgb(image, bgr="gri", stretch=1, Q=10, scale=None):
    """
    Create an RGB color composite image.

    Parameters
    ----------
    image : `MultibandExposure`
        `MultibandExposure` to display.
    bgr : sequence
        A 3-element sequence of filter names (i.e., keys of the exps dict)
        indicating what band to use for each channel. If `image` only has
        three filters then this parameter is ignored and the filters
        in the image are used.
    stretch: int
        The linear stretch of the image.
    Q: int
        The Asinh softening parameter.
    scale: list of 3 floats, each less than 1. (default: None)
        Re-scales the RGB channels.

    Returns
    -------
    rgb: ndarray
        RGB (integer, 8-bits per channel) colour image as an NxNx3 numpy array.
    """

    # If the image only has 3 bands, reverse the order of the bands
    #   to produce the RGB image
    if len(image) == 3:
        bgr = image.filters

    # Extract the primary image component of each Exposure with the
    #   .image property, and use .array to get a NumPy array view.

    if scale is None:
        r_im = image[bgr[2]].array  # numpy array for the r channel
        g_im = image[bgr[1]].array  # numpy array for the g channel
        b_im = image[bgr[0]].array  # numpy array for the b channel
    else:
        # manually re-scaling the images here
        r_im = image[bgr[2]].array * scale[0]
        g_im = image[bgr[1]].array * scale[1]
        b_im = image[bgr[0]].array * scale[2]

    rgb = make_lupton_rgb(image_r=r_im,
                          image_g=g_im,
                          image_b=b_im,
                          stretch=stretch, Q=Q)
    # "stretch" and "Q" are parameters to stretch and scale the pixel values

    return rgb

In [ ]:
def find_neighbourg(ra0,dec0,src):
    """ 
    """
    all_angles = []
    for index, row in src.iterrows():
        spherePointCenter = lsst.geom.SpherePoint(ra0*lsst.geom.degrees, dec0*lsst.geom.degrees)
        spt = lsst.geom.SpherePoint(row["coord_ra"]*lsst.geom.degrees, row["coord_dec"]*lsst.geom.degrees)
        ang = spherePointCenter.separation(spt)
        ang_arcsec = ang.asArcseconds()
        all_angles.append(ang_arcsec)
    return all_angles

## Get Pixel Scale

In [ ]:
import lsst.geom as geom
import lsst.sphgeom

skymap = butler.get('skyMap', skymap=skymapName, collections=collections )
tractInfo = skymap.generateTract(tract)
for patch in tractInfo:    
    patchID = patch.getSequentialIndex()
        
    ibb=patch.getInnerBBox()
    tWCS=tractInfo.getWcs()
       
    # loop on the 4 corners
    for icorn,corner in enumerate(ibb.getCorners()):
        p = geom.Point2D(corner.getX(), corner.getY())
        coord = tWCS.pixelToSky(p)

In [ ]:
tWCS

In [ ]:
#arcsec/pixel
pixel_scale = tWCS.getPixelScale().asArcseconds()

## Selected visits

In [ ]:
#inputfilename = "sources_objectTable-t3864-bg-o547-LATISS_runs_AUXTEL_DRP_IMAGING_20230509_20240414_w_2024_15_PREOPS-5069.csv"
inputfilename = "sources_objectTable-t3864-bg-o912-LATISS_runs_AUXTEL_DRP_IMAGING_20230509_20240414_w_2024_15_PREOPS-5069.csv"
df_myselectedvisits = pd.read_csv(inputfilename ,index_col=0)
tract = 3864
band = "g"
#objectname = 547
objectname = 912
path = f"calexp_t{tract}_b{band}_o{objectname}"
title = f"Auxtel Light Curves : tract = {tract}, band = {band}, object = {objectname} "
suptitle = inputfilename 

In [ ]:
if not os.path.exists(path):
    os.mkdir(path)

In [ ]:
df_myselectedvisits.sort_values("visit",inplace=True)
#df_myselectedvisits.sort_index(inplace=True)

In [ ]:
df_myselectedvisits

In [ ]:
ZPMAX = df_myselectedvisits["zeroPoint"].max()
ZPMAX

In [ ]:
df_myselectedvisits["attenuationFactor"] = np.power(10.,0.4*(ZPMAX-df_myselectedvisits["zeroPoint"]))

In [ ]:
df_myselectedvisits["attenuationFactor"]

In [ ]:
df_myselectedvisits["apFlux_50_0_instFlux_graycorr"] = df_myselectedvisits["apFlux_50_0_instFlux"]* df_myselectedvisits["attenuationFactor"]

In [ ]:
#psfMagDiffMmag

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(14,4))
df_myselectedvisits.plot.scatter(x="expMidptMjd",y="psfMag",ax=ax,s=20,c="zeroPoint",cmap="jet",grid=True,rot=45)
ax.set_title(title)
plt.suptitle(suptitle)
plt.tight_layout()
plt.show()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(14,4))
df_myselectedvisits.plot.scatter(x="expMidptMjd",y="apFlux_50_0_instFlux",ax=ax,s=20,c="zeroPoint",cmap="jet",grid=True,rot=45)
ax.set_title(title)
plt.suptitle(suptitle)
plt.tight_layout()
plt.show()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(6,6))
df_myselectedvisits.plot.scatter(x="zeroPoint",y="apFlux_50_0_instFlux",ax=ax,s=20,grid=True)
ax.set_title(title)
plt.suptitle(suptitle)
plt.tight_layout()
plt.show()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(6,6))
df_myselectedvisits.plot.scatter(x="zeroPoint",y="apFlux_50_0_instFlux_graycorr",ax=ax,s=20,grid=True)
ax.set_title(title)
plt.suptitle(suptitle)
plt.tight_layout()
plt.show()

In [ ]:
df_myselectedvisits[["visit","apFlux_50_0_instFlux_graycorr","apFlux_50_0_instFlux","localBackground_instFlux"]]

In [ ]:
df_myselectedvisits.min()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(14,4))
df_myselectedvisits.plot.scatter(x="expMidptMjd",y="psfMagDiffMmag",ax=ax,s=20,c="zeroPoint",cmap="jet",grid=True,rot=45)
ax.set_title(title)
plt.suptitle(suptitle)
plt.tight_layout()
plt.show()

In [ ]:
bundle_selected_object = df_myselectedvisits[["visit","psfMagDiffMmag"]]
fig, ax = plt.subplots(1,1,figsize=(16,2))
bundle_selected_object.plot.bar(x="visit",ax=ax,color="b" ,rot=90,grid=True)
title = f"psfMagDiffMmag for object {objectname} for tract {tract} in band {band} (AUXTEL)"
ax.set_title(title)

In [ ]:
print(path)

In [ ]:
listOfVisitId = df_myselectedvisits["visit"].values
listOfVisitId

In [ ]:
datasetType = 'calexp'

if objectname == 912:
    boxSize = 500
else:
    boxSize = 300
    
imMin, imMax, Q = -0.001, 0.004, 8
expMin, expMax = -25, 100



all_cutout = []
all_psfSigma = []
all_psfMagDiffMmag = []
all_psfMag = []
all_apFlux_50_0_instFlux = []                 
all_apFlux_50_0_instFluxErr = []  
all_neighbourgs = []
all_flag_deblend_skipped  = []

for index,visitId in enumerate(listOfVisitId):
    print(f"==================={index}) ====== {visitId} =============================")
    dataId = {'visit': visitId, 'instrument':instrument , 'detector': 0}
    calexp = butler.get('calexp', **dataId,collections=collections)
    src_cat = butler.get('sourceTable',**dataId, collections=collections)
    src_cat = src_cat[src_cat["extendedness"]==0]
    src_cat["psfMag"] = src_cat["psfFlux"].apply(convert_fluxtomag)
    src_cat = src_cat[src_cat["psfMag"]<16]
    
    
    print(f"Retrieved catalog of {len(src_cat)} sources.")
    row_source = df_myselectedvisits.iloc[index]

    # extract the source from the object for that visit
    x_target = row_source['x']
    y_target = row_source['y']
    ra_target= row_source['ra']
    dec_target= row_source['dec']
    psfMag  = row_source['psfMag']  
    psfSigma = row_source['psfSigma']
    psfMagDiffMmag = row_source['psfMagDiffMmag'] 
    apFlux_50_0_instFlux = row_source['apFlux_50_0_instFlux']
    apFlux_50_0_instFluxErr = row_source['apFlux_50_0_instFluxErr']

    # searcch the neighbouring stars
    separation = find_neighbourg(ra_target,dec_target,src_cat)
    src_cat["sep"] = separation
    if objectname == 547:
        cut_sep = ((src_cat["sep"] > 25) & (src_cat["sep"] < 27)) | (src_cat["sep"] < 1)
        #cut_sep = ( (src_cat["sep"] < 27)) | (src_cat["sep"] < 1)
    elif objectname == 912:
        cut_sep = (src_cat["sep"] < 46)
    src_cat = src_cat[cut_sep]
    neighbourgs = src_cat.sort_values(by="sep").iloc[:5]
    print(f"(target :: {x_target:.2f},{y_target:.2f}) , psfMag = {psfMag:.2f} mag, apFlux_50_0_instFlux={apFlux_50_0_instFlux:.0f} ADU" )
    print(neighbourgs[["x","y","sep","psfMag","apFlux_50_0_instFlux","ra","dec","deblend_skipped"]])

    flag_deblend_skipped = neighbourgs.iloc[0]["deblend_skipped"]
    
    
    
    
    if len(neighbourgs) < 2:
        continue
    
    all_neighbourgs.append(neighbourgs)

    xSrc = x_target
    ySrc = y_target
    
    minBbox = geom.Point2I(int(xSrc) - boxSize ,int(ySrc) - boxSize)
    maxBbox = geom.Point2I(int(xSrc) + boxSize, int(ySrc) + boxSize)
    srcBbox = geom.Box2I(minBbox, maxBbox)
    # Make the cutout
    try:
        subimg = afwImage.ExposureF(calexp, srcBbox, afwImage.PARENT, True)
        subimg2 = calexp.getImage().array[int(ySrc) - boxSize: int(ySrc) + boxSize+1, int(xSrc) - boxSize:int(xSrc) + boxSize+1 ]
        print(subimg2.shape) 
        subimgRgb = afwRgb.makeRGB(subimg.image.array, None, None, expMin, expMax - expMin, Q)
        #all_cutout.append(subimgRgb)
        all_cutout.append(subimg.image.array)
        #all_cutout.append(subimg2)
        all_psfSigma.append(psfSigma) 
        all_psfMag.append(psfMag)
        all_psfMagDiffMmag.append(psfMagDiffMmag)
        all_apFlux_50_0_instFlux.append(apFlux_50_0_instFlux)                
        all_apFlux_50_0_instFluxErr.append(apFlux_50_0_instFluxErr)  
        all_flag_deblend_skipped.append(flag_deblend_skipped)
    except Exception as inst:
        print(type(inst))    # the exception type
        print(inst.args)     # arguments stored in .args
        print(inst)     
        
   

In [ ]:
NIMG = len(all_cutout)
NCOLS = 4
NROWS = int(np.ceil(NIMG/NCOLS))

In [ ]:
from astropy.visualization import ZScaleInterval
z = ZScaleInterval()


In [ ]:
NIMG,NROWS 

In [ ]:
all_cutout[0].shape

In [ ]:
fig, axes = plt.subplots(ncols=NCOLS,nrows=NROWS,figsize=(6*NCOLS,6*NROWS))
for index,ax in enumerate(axes.flatten()):
    if index >= NIMG:
        break
    #cutout = all_cutout[index]
    cutout = all_cutout[index]
    z1,z2 = z.get_limits(cutout)
    visitId = listOfVisitId[index]
    #ax.imshow(cutout,interpolation="nearest")
    psfSigma = all_psfSigma[index]
    psfMagDiffMmag = all_psfMagDiffMmag[index]
    psfMag = all_psfMag[index]
    apFlux_50_0_instFlux = all_apFlux_50_0_instFlux[index]
    apFlux_50_0_instFluxErr = all_apFlux_50_0_instFluxErr[index]
    flag_deblend_skipped = all_flag_deblend_skipped[index]

    neighbourgs = all_neighbourgs[index]
    
    textstr = '\n'.join((
    r'$expos = %.0f$' % (visitId, ),
    r'$x_t = %.1f , y_t = %.1f$' % (x_target,y_target),
    r'$psfMag = %.02f mag$' %(psfMag, ), 
    r'$psfMagDiff = %.0f mmag$' %(psfMagDiffMmag, ),   
    r'$apFlux\_50\_instFlux = %.0f \;ADU$' %(apFlux_50_0_instFlux, ),  
    r'$apFlux\_50\_instFluxErr = %.0f \;ADU$' %(apFlux_50_0_instFluxErr, ),   
    r'$\sigma_{PSF} = %.2f pix$' % (psfSigma , ),
    r'$deb\_skip = %.0f $' % (flag_deblend_skipped  , )))
    ann = Annulus((boxSize,boxSize),r=5*psfSigma,width=0.1,color="red")
    ax.imshow(cutout,cmap="grey",vmin=z1, vmax=z2)
    ax.add_patch(ann)

    x_target = neighbourgs.iloc[0]["x"]
    y_target = neighbourgs.iloc[0]["y"]
    
    if len(neighbourgs) > 1:
        x_star = neighbourgs.iloc[1]["x"]
        y_star = neighbourgs.iloc[1]["y"]
        dx =   x_star -  x_target + boxSize
        dy =   y_star -  y_target + boxSize
        ann1 = Annulus((dx,dy),r=5*psfSigma,width=0.1,color="yellow")
        ax.add_patch(ann1)

    if len(neighbourgs) > 2:
        x_star = neighbourgs.iloc[2]["x"]
        y_star = neighbourgs.iloc[2]["y"]
        dx =   x_star -  x_target + boxSize
        dy =   y_star -  y_target + boxSize
        ann2 = Annulus((dx,dy),r=5*psfSigma,width=0.1,color="orange")
        ax.add_patch(ann2)
        
    if flag_deblend_skipped:
        ax.text(0.05, 0.95, textstr, transform=ax.transAxes, fontsize=16,verticalalignment='top',color="r", weight='bold',bbox=props) 
    else:
        ax.text(0.05, 0.95, textstr, transform=ax.transAxes, fontsize=16,verticalalignment='top',color="g", weight='bold',bbox=props) 
    title = f" {index}) {visitId}"
    ax.set_title(title)
plt.suptitle(suptitle,y=1.0,fontsize=16)
plt.tight_layout()
plt.show()

In [ ]:
neighbourgs["psfFlux"]

In [ ]:
all_ap50ratio = pd.DataFrame(columns=["visit","ratio","deblend_skipped","fl1","fl2"])
all_psfratio = pd.DataFrame(columns=["visit","ratio","deblend_skipped","fl1","fl2"])

for index in range(NIMG):
    visitId = listOfVisitId[index]
    psfSigma = all_psfSigma[index]
    psfMagDiffMmag = all_psfMagDiffMmag[index]
    psfMag = all_psfMag[index]
    apFlux_50_0_instFlux = all_apFlux_50_0_instFlux[index]
    apFlux_50_0_instFluxErr = all_apFlux_50_0_instFluxErr[index]

    neighbourgs = all_neighbourgs[index]
    #print(neighbourgs[["x","y","sep","psfMag","apFlux_50_0_instFlux","ra","dec"]])
    ratio_ap = neighbourgs.iloc[1]["apFlux_50_0_instFlux"]/neighbourgs.iloc[0]["apFlux_50_0_instFlux"]
    ratio_psf = neighbourgs.iloc[1]["psfFlux"]/neighbourgs.iloc[0]["psfFlux"]
 
    all_ap50ratio.loc[index] = [visitId,ratio_ap,neighbourgs.iloc[0]["deblend_skipped"],neighbourgs.iloc[0]["apFlux_50_0_instFlux"],neighbourgs.iloc[1]["apFlux_50_0_instFlux"] ] 
    all_psfratio.loc[index] = [visitId,ratio_psf,neighbourgs.iloc[0]["deblend_skipped"],neighbourgs.iloc[0]["psfFlux"],neighbourgs.iloc[1]["psfFlux"]] 
    

#### Computing the relative ratio

##### Method 1

In [ ]:
all_ap50ratio["relratio"]= all_ap50ratio["ratio"]/all_ap50ratio["ratio"].mean()
all_psfratio["relratio"]= all_psfratio["ratio"]/all_psfratio["ratio"].mean()

##### Method 2

In [ ]:
all_ap50ratio["relratio"]= all_ap50ratio["ratio"]* all_ap50ratio["fl1"].mean()/all_ap50ratio["fl2"].mean()
all_psfratio["relratio"]= all_psfratio["ratio"]* all_psfratio["fl1"].mean()/all_psfratio["fl2"].mean()

In [ ]:
clippedAp50Array = scipy.stats.sigmaclip(all_ap50ratio["relratio"].values, low=3.0, high=3.0).clipped
clippedAp50Mean = np.mean(clippedAp50Array)
clippedAp50Std = np.std(clippedAp50Array)
clippedpsfArray = scipy.stats.sigmaclip(all_psfratio["relratio"].values, low=3.0, high=3.0).clipped
clippedpsfMean = np.mean(clippedpsfArray)
clippedpsfStd = np.std(clippedpsfArray)
xmin = 1 - 5*clippedAp50Std 
xmax = 1 + 5*clippedAp50Std 

In [ ]:
std_ap50 = clippedAp50Std *1000
std_psf = clippedpsfStd *1000
label_ap50 = f"ap50: $\sigma$ = {std_ap50:.1f} mmag"
label_psf = f"psf: $\sigma$ = {std_psf:.1f} mmag"

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(8,8))

all_ap50ratio["relratio"].hist(bins=20,range=(xmin,xmax),ax=ax,color="b",label=label_ap50,histtype="step",lw=3)
all_ap50ratio["relratio"][all_ap50ratio["deblend_skipped"]].hist(bins=20,range=(xmin,xmax),ax=ax,color="grey",histtype="step",label="ap50 : deblend detected")
all_psfratio["relratio"].hist(bins=20,range=(xmin,xmax),ax=ax,color="r",label=label_psf ,histtype="step",lw=3)
all_psfratio["relratio"][all_psfratio["deblend_skipped"]].hist(bins=20,range=(xmin,xmax),ax=ax,color="k",histtype="step",label="psf : deblend detected")
ax.set_title("relative flux ratio ")
ax.set_xlabel("relative flux ratio ")
ax.set_xlim(xmin,xmax)
ax.legend()
plt.suptitle(suptitle)

In [ ]:
#src_cat.getSchema().getNames(topOnly=True)

In [ ]:
#columns = src_cat.schema.getNames()
#columns

In [ ]:
#src_cat.getSchema().find('id')